In [255]:
# import panda 
import pandas as pd

# load the csv file 
df = pd.read_csv('../raw/relationship.csv')

# print the first 5 rows of the dataframe
df.head()
# update the name_a with split string, get the first value
# update the name_b with split string, get the second value
df['name_a'] = df['name_a'].str.split(' ').str[0]
df['name_b'] = df['name_b'].str.split(' ').str[0] 

df_list = []

# loop through all the unique values in the name_a column
for name in df['name_a'].unique():
    # get the rows where the name_a or name_b is equal to the name
    df_name = df[(df['name_a'] == name) | (df['name_b'] == name)]

    # swap the name_a and name_b if the name_b is equal to the name
    df_name.loc[df_name['name_b'] == name, ['name_a', 'name_b']] = df_name.loc[df_name['name_b'] == name, ['name_b', 'name_a']].values
    # append the dataframe to the list
    df_list.append(df_name)

# concat the list of dataframes into one dataframe
df_total = pd.concat(df_list)

# drop the duplicates if name_a and name_b are the same
df_total = df_total.drop_duplicates(subset=['name_a', 'name_b'])

# drop the duplicated _index 
#df_total = df_total.drop_duplicates(subset=['_index'])

df_total.to_csv('../processed/relationship.csv', index=False)


c:\Users\k-main\anaconda3\lib\site-packages\pandas\core\indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [256]:

# melt the value with the unique value into new columes



In [264]:
explored = []

json ={}

# loop through all the unique values in the name_a column
for name in df_total['name_a'].unique():
    json[name] = {}

# iterate through the dataframe
# loop through all the unique values in the name_a column
for name in df_total['name_a'].unique():
    # if the name is in the explored list, skip
    if name in explored:
        continue
    # get the rows where the name_a and append all the name_b to as the json value
    temp_list = list(df_total[df_total['name_a'] == name]['name_b'].values)
    # remove the entry if it exit in the exlored list
    temp_list = [x for x in temp_list if x not in explored]
    # append the name to the explored list
    explored.append(name)
    json[name] = {"name": name, "children": temp_list,"relations":temp_list.__len__()}


# sort the json by the relations
json = {k: v for k, v in sorted(json.items(), key=lambda item: item[1]['relations'], reverse=True)}


the_explored_list = []

def gen_json(name, input_json):
    # loop through the json and recursively call the function until the children is empty
    temp_json = {}
    temp_json["name"] = input_json[name]["name"]
    temp_json["children"] = []

    # first check if all the keys in the children is in the json
    json[name]["children"] = [x for x in json[name]["children"] if x in json.keys()]
    # if does not exist, create the entry in the original json 
    for child in json[name]["children"]:
        if child not in input_json.keys():
            input_json[child] = {"name": child, "children": [], "relations": 0}
    

    # sort the json children by the relations
    json[name]['children'] = sorted(json[name]['children'], key=lambda x: json[x]['relations'], reverse=True)
    the_explored_list.append(name)



    # loop through the children
    for child in json[name]["children"]:
        # if the child is in the json, call the function again
        # check if the key is in the json
        if child in the_explored_list:
            continue
        if child in input_json.keys():
            temp_json["children"].append(gen_json(child, input_json))
        else:
            temp_json["children"].append({"name": child})
        # iterate the children
        #temp_json["children"].append(gen_json(child, input_json))
    return temp_json


output_json = gen_json("周进",json)

# write the json to the file
import json
with open('../processed/relationship.json', 'w') as outfile:
    json.dump(output_json, outfile, ensure_ascii=False)



KeyError: 'children'

In [183]:
import json


def get_relationship_json(character_name,start_index,end_index,upfront,df_input,explored_names=[]):
    # start constructing the json object
    json_object = {}
    json_object['name'] = character_name
    json_object["children"] = []
    # get the rows where the name_a or name_b is equal to the name and the _index equals to the start_index
    df_name = df_input[((df_input['name_a'] == character_name) | (df_input['name_b'] == character_name)) & (df_input['chapter'] >= start_index) & (df_input['chapter'] < start_index + upfront) & (df_input['chapter'] < end_index)]
    # loop through all the rows and append the name_a or name_b to the children list
    #print (df_name)
    


    for index, row in df_name.iterrows():
            if row['name_a'] == character_name and row['name_b'] not in explored_names:
                json_object["children"].append(get_relationship_json(row['name_b'],row['chapter']+1,end_index,upfront,df_input))
                explored_names.append(character_name)
            elif row['name_b'] == character_name and row['name_a'] not in explored_names:
                json_object["children"].append(get_relationship_json(row['name_a'],row['chapter']+1,end_index,upfront,df_input))
                explored_names.append(character_name)
    
    if (json_object['children'] == []):
        # remove the children key if the children list is empty
        json_object.pop("children", None)
    
    
    #print (json_object)
    return json_object



output_json = get_relationship_json("周进",0,20,8,df_total)

# save the json object to a file
with open('../processed/relationship.json', 'w') as outfile:
    json.dump(output_json, outfile)


In [184]:
output_json

{'name': '周进',
 'children': [{'name': '夏总甲'},
  {'name': '梅玖', 'children': [{'name': '范进'}, {'name': '范进'}]},
  {'name': '金有馀'},
  {'name': '范进',
   'children': [{'name': '严贡生', 'children': [{'name': '严监生'}]},
    {'name': '汤奉'},
    {'name': '荀玫'},
    {'name': '蘧景玉',
     'children': [{'name': '王惠'},
      {'name': '蘧公孙',
       'children': [{'name': '鲁编修',
         'children': [{'name': '景兰江'}, {'name': '景兰江'}]},
        {'name': '马纯上',
         'children': [{'name': '洪憨仙'},
          {'name': '匡超人',
           'children': [{'name': '李本瑛'},
            {'name': '景兰江'},
            {'name': '潘三'},
            {'name': '景兰江'}]}]}]}]},
    {'name': '牛布衣'},
    {'name': '荀玫'}]},
  {'name': '荀玫'},
  {'name': '荀玫'}]}